Develop a program to find the GNF of the given CFG

In [ ]:
# 1b

## Author : Arjun ##

class CFG:
    def __init__(self, start_symbol, productions):
        self.start_symbol = start_symbol
        self.productions = productions
        self.new_symbol_count = 0

    def get_new_symbol(self):
        self.new_symbol_count += 1
        return f"X{self.new_symbol_count}"

    def convert_to_gnf(self):
        self.eliminate_null_productions()
        self.eliminate_unit_productions()
        self.convert_to_gnf_form()

    def eliminate_null_productions(self):
        nullable = set()
        for lhs, rules in self.productions.items():
            for rule in rules:
                if rule == ['']:
                    nullable.add(lhs)

        for lhs in list(self.productions):
            new_rules = set()
            rules = self.productions[lhs]
            for rule in rules:
                if rule != ['']:
                    positions = [i for i, sym in enumerate(rule) if sym in nullable]
                    for i in range(1 << len(positions)):
                        new_rule = list(rule)
                        for pos in positions:
                            if i & (1 << positions.index(pos)):
                                new_rule[pos] = None
                        new_rule = tuple(sym for sym in new_rule if sym is not None)
                        if new_rule:
                            new_rules.add(new_rule)
            self.productions[lhs] = list(new_rules)

    def eliminate_unit_productions(self):
        units = {lhs: set() for lhs in self.productions}
        for lhs, rules in self.productions.items():
            for rule in rules:
                if len(rule) == 1 and rule[0].isupper():
                    units[lhs].add(rule[0])

        changes = True
        while changes:
            changes = False
            for lhs, targets in units.items():
                new_targets = set(targets)
                for target in targets:
                    new_targets |= units[target]
                if new_targets != targets:
                    changes = True
                    units[lhs] = new_targets

        for lhs, targets in units.items():
            self.productions[lhs] = [rule for rule in self.productions[lhs] if len(rule) != 1 or not rule[0].isupper()]
            for target in targets:
                if target != lhs:
                    self.productions[lhs].extend(self.productions[target])

    def convert_to_gnf_form(self):
        for lhs, rules in list(self.productions.items()):
            new_rules = []
            for rule in rules:
                if rule[0].islower() or len(rule) == 1:  # If starts with a terminal or is a single non-terminal
                    new_rules.append(rule)
                else:  # Need to rearrange to form a Bα where α starts with a terminal
                    new_symbol = self.get_new_symbol()
                    self.productions[new_symbol] = [rule[1:]]  # Create a new production for the rest
                    new_rules.append([rule[0], new_symbol])  # Modify the original to include the new symbol
            self.productions[lhs] = new_rules

# Example usage
productions = {
    'S': [['A', 'B'], ['a']],
    'A': [['C', 'D'], ['a']],
    'B': [['b'], ['S']],
    'C': [['c']],
    'D': [['d']]
}
cfg = CFG('S', productions)
cfg.convert_to_gnf()
for lhs, rules in cfg.productions.items():
    print(f"{lhs} -> {[' '.join(rule) for rule in rules]}")

Develop a program to check the equivalence of two finite automata

In [ ]:
# 2b

## Author : Arjun ##

class FiniteAutomaton:
    def __init__(self, states, alphabet, transitions, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.start_state = start_state
        self.accept_states = accept_states

    def is_equivalent_to(self, other):
        # Create product automaton states
        product_states = {(p, q) for p in self.states for q in other.states}
        product_start_state = (self.start_state, other.start_state)
        product_accept_states = {(p, q) for p in self.accept_states for q in other.accept_states}
        product_transitions = {}

        # Build transitions for the product automaton
        for (p, q) in product_states:
            for a in self.alphabet:
                if a in self.transitions.get(p, {}) and a in other.transitions.get(q, {}):
                    next_p = self.transitions[p][a]
                    next_q = other.transitions[q][a]
                    if (p, q) not in product_transitions:
                        product_transitions[(p, q)] = {}
                    product_transitions[(p, q)][a] = (next_p, next_q)

        # Perform BFS to check for reachability of distinguishing states
        from collections import deque
        queue = deque([product_start_state])
        visited = set([product_start_state])

        while queue:
            current = queue.popleft()
            if (current[0] in self.accept_states) != (current[1] in other.accept_states):
                return False  # Distinguishing state found

            for a in self.alphabet:
                if a in product_transitions.get(current, {}):
                    next_state = product_transitions[current][a]
                    if next_state not in visited:
                        visited.add(next_state)
                        queue.append(next_state)

        return True  # No distinguishing states found

# Example usage
fa1 = FiniteAutomaton(
    states={'q0', 'q1'},
    alphabet={'0', '1'},
    transitions={
        'q0': {'0': 'q0', '1': 'q1'},
        'q1': {'0': 'q1', '1': 'q0'}
    },
    start_state='q0',
    accept_states={'q1'}
)

fa2 = FiniteAutomaton(
    states={'p0', 'p1'},
    alphabet={'0', '1'},
    transitions={
        'p0': {'0': 'p0', '1': 'p1'},
        'p1': {'0': 'p1', '1': 'p0'}
    },
    start_state='p0',
    accept_states={'p1'}
)

print("The two automata are equivalent" if fa1.is_equivalent_to(fa2) else "The two automata are not equivalent.")

Develop a program to simplify the given context free grammar

In [ ]:
# 3b

## Author : Arjun ##

class CFG:
    def __init__(self, start_symbol, productions):
        self.start_symbol = start_symbol
        self.productions = {k: [list(r) if isinstance(r, list) else r for r in v] for k, v in productions.items()}

    def remove_useless_symbols(self):
        productive = set()
        changes = True
        while changes:
            changes = False
            for lhs, rules in self.productions.items():
                for rule in rules:
                    if all(symbol in productive or not symbol.isupper() for symbol in rule):
                        if lhs not in productive:
                            productive.add(lhs)
                            changes = True

        reachable = set([self.start_symbol])
        changes = True
        while changes:
            changes = False
            new_reachable = reachable.copy()
            for symbol in reachable:
                for rule in self.productions.get(symbol, []):
                    for char in rule:
                        if char.isupper() and char not in reachable:
                            new_reachable.add(char)
                            changes = True
            reachable = new_reachable

        useful = productive & reachable
        new_productions = {k: [r for r in v if all(sym in useful or not sym.isupper() for sym in r)]
                           for k, v in self.productions.items() if k in useful}
        self.productions = new_productions

    def eliminate_null_productions(self):
        nullable = set()
        for lhs, rules in self.productions.items():
            for rule in rules:
                if rule == ['']:
                    nullable.add(lhs)

        changes = True
        while changes:
            changes = False
            for lhs, rules in self.productions.items():
                for rule in rules:
                    if any(sym in nullable for sym in rule) and lhs not in nullable:
                        nullable.add(lhs)
                        changes = True

        new_productions = {}
        for lhs, rules in self.productions.items():
            new_rules = set()
            for rule in rules:
                if rule != ['']:
                    new_rules.add(tuple(rule))
                    positions = [i for i, sym in enumerate(rule) if sym in nullable]
                    for i in range(1, 1 << len(positions)):
                        new_rule = list(rule)
                        for pos in positions:
                            if i & (1 << positions.index(pos)):
                                new_rule[pos] = ''
                        new_rule = tuple(sym for sym in new_rule if sym != '')
                        new_rules.add(new_rule)
            new_productions[lhs] = [list(r) for r in new_rules]
        self.productions = new_productions

    def remove_unit_productions(self):
        new_productions = {k: [] for k in self.productions}
        for lhs, rules in self.productions.items():
            direct_units = {lhs}
            units = set()
            while direct_units:
                new_units = set()
                for unit in direct_units:
                    for rule in self.productions[unit]:
                        if len(rule) == 1 and rule[0].isupper():
                            if rule[0] not in units:
                                new_units.add(rule[0])
                        else:
                            new_productions[lhs].append(rule)
                units.update(new_units)
                direct_units = new_units
        self.productions = new_productions

    def simplify(self):
        self.remove_useless_symbols()
        self.eliminate_null_productions()
        self.remove_unit_productions()
        self.remove_useless_symbols()

    def print_productions(self):
        for lhs, rules in self.productions.items():
            formatted_rules = [' '.join(rule) if rule else 'ε' for rule in rules]
            print(f"{lhs} -> {' | '.join(formatted_rules)}")

# Example usage
productions = {
    'S': [['A', 'b'], ['A']],
    'A': [['a'], [''], ['S']],
    'B': [['b']]
}
cfg = CFG('S', productions)
cfg.simplify()
cfg.print_productions()


Develop a program to find the CNF of the given CFG.

In [ ]:
# 4b

## Author : Arjun ##

class CFG:
    def __init__(self, start_symbol, productions):
        self.start_symbol = start_symbol
        self.productions = productions
        self.new_symbol_count = 0

    def get_new_symbol(self):
        self.new_symbol_count += 1
        return f"X{self.new_symbol_count}"

    def convert_to_cnf(self):
        self.ensure_start_symbol()
        self.eliminate_null_productions()
        self.eliminate_unit_productions()
        self.eliminate_long_productions()
        self.replace_terminals_in_nonterminal_rules()

    def ensure_start_symbol(self):
        # Create a new start symbol and add a production from new start to old start if necessary
        if any(self.start_symbol in rule for rules in self.productions.values() for rule in rules):
            new_start = self.get_new_symbol()
            self.productions[new_start] = [[self.start_symbol]]
            self.start_symbol = new_start

    def eliminate_null_productions(self):
        nullable = set()
        for lhs, rules in self.productions.items():
            for rule in rules:
                if rule == ['']:
                    nullable.add(lhs)

        for lhs in list(self.productions):
            new_rules = set()
            rules = self.productions[lhs]
            for rule in rules:
                if rule != ['']:
                    positions = [i for i, sym in enumerate(rule) if sym in nullable]
                    for i in range(1 << len(positions)):
                        new_rule = list(rule)
                        for pos in positions:
                            if i & (1 << positions.index(pos)):
                                new_rule[pos] = None
                        new_rule = tuple(sym for sym in new_rule if sym is not None)
                        if new_rule:
                            new_rules.add(new_rule)
            self.productions[lhs] = list(new_rules)

    def eliminate_unit_productions(self):
        units = {lhs: set() for lhs in self.productions}
        for lhs, rules in self.productions.items():
            for rule in rules:
                if len(rule) == 1 and rule[0].isupper():
                    units[lhs].add(rule[0])

        changes = True
        while changes:
            changes = False
            for lhs, targets in units.items():
                new_targets = set(targets)
                for target in targets:
                    new_targets |= units[target]
                if new_targets != targets:
                    changes = True
                    units[lhs] = new_targets

        for lhs, targets in units.items():
            self.productions[lhs] = [rule for rule in self.productions[lhs] if len(rule) != 1 or not rule[0].isupper()]
            for target in targets:
                if target != lhs:
                    self.productions[lhs].extend(self.productions[target])

    def eliminate_long_productions(self):
        for lhs, rules in list(self.productions.items()):
            new_rules = []
            for rule in rules:
                while len(rule) > 2:
                    first_symbol = rule[-2]
                    second_symbol = rule[-1]
                    new_symbol = self.get_new_symbol()
                    self.productions[new_symbol] = [[first_symbol, second_symbol]]
                    rule = rule[:-2] + [new_symbol]
                new_rules.append(rule)
            self.productions[lhs] = new_rules

    def replace_terminals_in_nonterminal_rules(self):
        for lhs, rules in list(self.productions.items()):
            new_rules = []
            for rule in rules:
                new_rule = []
                for symbol in rule:
                    if symbol.islower():  # It's a terminal
                        new_symbol = self.get_new_symbol()
                        self.productions[new_symbol] = [[symbol]]
                        new_rule.append(new_symbol)
                    else:
                        new_rule.append(symbol)
                new_rules.append(new_rule)
            self.productions[lhs] = new_rules

# Example usage
productions = {
    'S': [['A', 'B'], ['a']],
    'A': [['C', 'D'], ['']],
    'B': [['b'], ['E']],
    'C': [['c']],
    'D': [['d']],
    'E': [['e']]
}
cfg = CFG('S', productions)
cfg.convert_to_cnf()
for lhs, rules in cfg.productions.items():
    print(f"{lhs} -> {[' '.join(rule) for rule in rules]}")

Develop a program to minimize the given finite automata

In [ ]:
class DFA:
    def __init__(self, states, alphabet, transition_function, start_state, final_states):
        self.states = set(states)
        self.alphabet = set(alphabet)
        self.transition_function = transition_function  # Expected as a dictionary {(state, symbol): next_state}
        self.start_state = start_state
        self.final_states = set(final_states)
        self.state_names = {}  # Mapping of frozenset state groups to named states

    def minimize(self):
        self.remove_unreachable_states()
        return self.partition_refinement()

    def remove_unreachable_states(self):
        reachable = set()
        stack = [self.start_state]
        while stack:
            state = stack.pop()
            if state not in reachable:
                reachable.add(state)
                for symbol in self.alphabet:
                    next_state = self.transition_function.get((state, symbol))
                    if next_state and next_state not in reachable:
                        stack.append(next_state)
        self.states = reachable
        self.final_states = {state for state in self.final_states if state in reachable}

    def partition_refinement(self):
        P = {frozenset(self.final_states), frozenset(self.states - self.final_states)}
        W = P.copy()
        while W:
            A = W.pop()
            for c in self.alphabet:
                X = {s for s in self.states if self.transition_function.get((s, c)) in A}
                new_partitions = set()
                for Y in P:
                    intersection = X & Y
                    difference = Y - X
                    if intersection and difference:
                        new_partitions.update([frozenset(intersection), frozenset(difference)])
                        if Y in W:
                            W.remove(Y)
                            W.update([frozenset(intersection), frozenset(difference)])
                        else:
                            W.add(frozenset(intersection) if len(intersection) <= len(difference) else frozenset(difference))
                    else:
                        new_partitions.add(Y)
                P = new_partitions
        return self.reconstruct_dfa(P)

    def reconstruct_dfa(self, partitions):
        new_states = {state_group: f'S{index}' for index, state_group in enumerate(partitions)}
        self.state_names = new_states
        new_start_state = next(new_states[state_group] for state_group in partitions if self.start_state in state_group)
        new_final_states = {new_states[state_group] for state_group in partitions if state_group & self.final_states}
        new_transition_function = {}
        for state_group, new_state in new_states.items():
            representative = next(iter(state_group))
            for symbol in self.alphabet:
                target_group = next((sg for sg in partitions if self.transition_function.get((representative, symbol)) in sg), None)
                if target_group is not None:
                    new_transition_function[(new_state, symbol)] = new_states[target_group]
        return DFA(new_states.values(), self.alphabet, new_transition_function, new_start_state, new_final_states)

def display_dfa(dfa):
    print("\nMinimized DFA:")
    print("States:", list(dfa.states))
    print("Alphabet:", dfa.alphabet)
    print("Start State:", dfa.start_state)
    print("Final States:", list(dfa.final_states))
    print("Transition Function:")
    for (state, symbol), next_state in dfa.transition_function.items():
        print(f"  ({state}, {symbol}) -> {next_state}")

# Example of DFA creation and minimization
states = ['q0', 'q1', 'q2', 'q3']
alphabet = ['0', '1']
transition_function = {
    ('q0', '0'): 'q1', ('q0', '1'): 'q0',
    ('q1', '0'): 'q2', ('q1', '1'): 'q0',
    ('q2', '0'): 'q2', ('q2', '1'): 'q3',
    ('q3', '0'): 'q2', ('q3', '1'): 'q3',
}
start_state = 'q0'
final_states = ['q2']

dfa = DFA(states, alphabet, transition_function, start_state, final_states)
minimized_dfa = dfa.minimize()

# Display the minimized DFA
display_dfa(minimized_dfa)


Develop a program to convert the given Mealy machine to Moore machine

In [ ]:
# 6b

## Author : Arjun ##

class MealyMachine:
    def __init__(self, states, input_alphabet, output_alphabet, transition_function, output_function):
        self.states = states
        self.input_alphabet = input_alphabet
        self.output_alphabet = output_alphabet
        self.transition_function = transition_function
        self.output_function = output_function

class MooreMachine:
    def __init__(self, states, input_alphabet, output_alphabet, transition_function, state_output):
        self.states = states
        self.input_alphabet = input_alphabet
        self.output_alphabet = output_alphabet
        self.transition_function = transition_function
        self.state_output = state_output

def convert_mealy_to_moore(mealy):
    new_states = {}
    new_transition_function = {}
    new_state_output = {}

    # Generate new states and output mapping
    for state in mealy.states:
        for input_symbol in mealy.input_alphabet:
            output = mealy.output_function[(state, input_symbol)]
            new_state = (state, output)
            new_states[new_state] = True
            new_state_output[new_state] = output

    # Generate new transition function
    for state in mealy.states:
        for input_symbol in mealy.input_alphabet:
            original_target_state = mealy.transition_function[(state, input_symbol)]
            output = mealy.output_function[(state, input_symbol)]
            new_origin_state = (state, output)
            new_target_state = (original_target_state, mealy.output_function[(original_target_state, input_symbol)])
            new_transition_function[(new_origin_state, input_symbol)] = new_target_state

    # Create Moore machine
    return MooreMachine(list(new_states.keys()), mealy.input_alphabet, mealy.output_alphabet, new_transition_function, new_state_output)

def display_moore_machine(moore):
    print("\nMoore Machine States:")
    for state in moore.states:
        print(f"  State: {state}, Output: {moore.state_output[state]}")

    print("\nMoore Machine Transition Function:")
    for (state, input_symbol), target_state in moore.transition_function.items():
        print(f"  Transition: ({state}, {input_symbol}) -> {target_state}")

# Example usage
mealy_states = ['A', 'B']
mealy_input_alphabet = ['0', '1']
mealy_output_alphabet = ['x', 'y']
mealy_transition_function = {('A', '0'): 'B', ('A', '1'): 'A', ('B', '0'): 'A', ('B', '1'): 'B'}
mealy_output_function = {('A', '0'): 'x', ('A', '1'): 'y', ('B', '0'): 'y', ('B', '1'): 'x'}

mealy = MealyMachine(mealy_states, mealy_input_alphabet, mealy_output_alphabet, mealy_transition_function, mealy_output_function)
moore = convert_mealy_to_moore(mealy)

display_moore_machine(moore)


Develop a program to convert the given Moore machine to Mealy machine

In [ ]:
# 7b

## Author : Arjun ##

class NFA:
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.start_state = start_state
        self.accept_states = accept_states

    def epsilon_closure(self, states):
        stack = list(states)
        closure = set(states)
        while stack:
            state = stack.pop()
            if state in self.transition_function and '' in self.transition_function[state]:
                for next_state in self.transition_function[state]['']:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def move(self, states, symbol):
        result = set()
        for state in states:
            if state in self.transition_function and symbol in self.transition_function[state]:
                result.update(self.transition_function[state][symbol])
        return result

class DFA:
    def __init__(self):
        self.states = set()
        self.transition_function = {}
        self.start_state = None
        self.accept_states = set()
        self.state_names = {}

    def name_state(self, state_set):
        if state_set not in self.state_names:
            name = f"S{len(self.state_names)}"
            self.state_names[state_set] = name
            return name
        return self.state_names[state_set]

def convert_nfa_to_dfa(nfa):
    dfa = DFA()
    unmarked_states = []
    dfa_start_state_set = frozenset(nfa.epsilon_closure({nfa.start_state}))
    dfa.start_state = dfa_start_state_set
    dfa.states.add(dfa_start_state_set)
    unmarked_states.append(dfa_start_state_set)

    while unmarked_states:
        current_dfa_state = unmarked_states.pop()
        for symbol in nfa.alphabet:
            if symbol == '':
                continue  # Ignore epsilon transitions for DFA
            next_state_set = nfa.epsilon_closure(nfa.move(current_dfa_state, symbol))
            if not next_state_set:
                continue
            next_state_set = frozenset(next_state_set)
            if next_state_set not in dfa.states:
                dfa.states.add(next_state_set)
                unmarked_states.append(next_state_set)
            if current_dfa_state not in dfa.transition_function:
                dfa.transition_function[current_dfa_state] = {}
            dfa.transition_function[current_dfa_state][symbol] = next_state_set
            if any(state in nfa.accept_states for state in next_state_set):
                dfa.accept_states.add(next_state_set)

    return dfa

# Example usage
nfa = NFA(
    states={'q0', 'q1', 'q2'},
    alphabet={'0', '1'},
    transition_function={
        'q0': {'': {'q1'}, '0': {'q1'}, '1': {'q2'}},
        'q1': {'0': {'q2'}, '1': {'q0'}},
        'q2': {'': {'q0'}}
    },
    start_state='q0',
    accept_states={'q2'}
)

dfa = convert_nfa_to_dfa(nfa)

# Naming states for clarity
for state_set in dfa.states:
    dfa.name_state(state_set)

# Print the converted DFA with clearer state names
print("DFA States:", [dfa.name_state(state) for state in dfa.states])
print("DFA Start State:", dfa.name_state(dfa.start_state))
print("DFA Accept States:", [dfa.name_state(state) for state in dfa.accept_states])
for state, transitions in dfa.transition_function.items():
    for symbol, next_state in transitions.items():
        print(f"Transition from {dfa.name_state(state)} on '{symbol}' goes to {dfa.name_state(next_state)}")

Develop a program to convert the given NFA to DFA

In [ ]:
# 8b

## Author : Arjun ##

class NFA:
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.start_state = start_state
        self.accept_states = accept_states

    def epsilon_closure(self, states):
        stack = list(states)
        closure = set(states)
        while stack:
            state = stack.pop()
            if state in self.transition_function and '' in self.transition_function[state]:
                for next_state in self.transition_function[state]['']:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def move(self, states, symbol):
        result = set()
        for state in states:
            if state in self.transition_function and symbol in self.transition_function[state]:
                result.update(self.transition_function[state][symbol])
        return result

class DFA:
    def __init__(self):
        self.states = set()
        self.transition_function = {}
        self.start_state = None
        self.accept_states = set()
        self.state_names = {}

    def name_state(self, state_set):
        if state_set not in self.state_names:
            name = f"S{len(self.state_names)}"
            self.state_names[state_set] = name
            return name
        return self.state_names[state_set]

def convert_nfa_to_dfa(nfa):
    dfa = DFA()
    unmarked_states = []
    dfa_start_state_set = frozenset(nfa.epsilon_closure({nfa.start_state}))
    dfa.start_state = dfa_start_state_set
    dfa.states.add(dfa_start_state_set)
    unmarked_states.append(dfa_start_state_set)

    while unmarked_states:
        current_dfa_state = unmarked_states.pop()
        for symbol in nfa.alphabet:
            if symbol == '':
                continue  # Ignore epsilon transitions for DFA
            next_state_set = nfa.epsilon_closure(nfa.move(current_dfa_state, symbol))
            if not next_state_set:
                continue
            next_state_set = frozenset(next_state_set)
            if next_state_set not in dfa.states:
                dfa.states.add(next_state_set)
                unmarked_states.append(next_state_set)
            if current_dfa_state not in dfa.transition_function:
                dfa.transition_function[current_dfa_state] = {}
            dfa.transition_function[current_dfa_state][symbol] = next_state_set
            if any(state in nfa.accept_states for state in next_state_set):
                dfa.accept_states.add(next_state_set)

    return dfa

# Example usage
nfa = NFA(
    states={'q0', 'q1', 'q2'},
    alphabet={'0', '1'},
    transition_function={
        'q0': {'': {'q1'}, '0': {'q1'}, '1': {'q2'}},
        'q1': {'0': {'q2'}, '1': {'q0'}},
        'q2': {'': {'q0'}}
    },
    start_state='q0',
    accept_states={'q2'}
)

dfa = convert_nfa_to_dfa(nfa)

# Naming states for clarity
for state_set in dfa.states:
    dfa.name_state(state_set)

# Print the converted DFA with clearer state names
print("DFA States:", [dfa.name_state(state) for state in dfa.states])
print("DFA Start State:", dfa.name_state(dfa.start_state))
print("DFA Accept States:", [dfa.name_state(state) for state in dfa.accept_states])
for state, transitions in dfa.transition_function.items():
    for symbol, next_state in transitions.items():
        print(f"Transition from {dfa.name_state(state)} on '{symbol}' goes to {dfa.name_state(next_state)}")

Develop a program to convert the given NFA with epsilon moves to NFA without epsilon moves

In [ ]:
# 9b

## Author : Arjun ##

class NFA:
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.start_state = start_state
        self.accept_states = accept_states

    def epsilon_closure(self, state):
        closure = {state}
        stack = [state]
        while stack:
            current = stack.pop()
            if current in self.transition_function and '' in self.transition_function[current]:
                for next_state in self.transition_function[current]['']:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def convert_to_nfa_without_epsilon(self):
        new_transition_function = {}
        new_accept_states = set()

        for state in self.states:
            closure = self.epsilon_closure(state)
            # Update accept states based on epsilon closure
            if any(s in self.accept_states for s in closure):
                new_accept_states.add(state)

            # Compile transitions from epsilon closure
            for input_symbol in self.alphabet:
                next_states = set()
                for s in closure:
                    if s in self.transition_function and input_symbol in self.transition_function[s]:
                        next_states.update(self.transition_function[s][input_symbol])
                if next_states:
                    if state not in new_transition_function:
                        new_transition_function[state] = {}
                    new_transition_function[state][input_symbol] = next_states

        return NFA(self.states, self.alphabet, new_transition_function, self.start_state, new_accept_states)

# Example usage
nfa_with_epsilon = NFA(
    states={'q0', 'q1', 'q2', 'q3'},
    alphabet={'a', 'b'},
    transition_function={
        'q0': {'': {'q1', 'q2'}},
        'q1': {'a': {'q1'}, 'b': {'q2'}},
        'q2': {'a': {'q3'}},
        'q3': {'b': {'q1'}}
    },
    start_state='q0',
    accept_states={'q3'}
)

nfa_without_epsilon = nfa_with_epsilon.convert_to_nfa_without_epsilon()

# Output the converted NFA
print("States:", nfa_without_epsilon.states)
print("Start State:", nfa_without_epsilon.start_state)
print("Accept States:", nfa_without_epsilon.accept_states)
for state, transitions in nfa_without_epsilon.transition_function.items():
    for symbol, next_states in transitions.items():
        print(f"Transition from {state} on '{symbol}' goes to {next_states}")

Develop a program to check the acceptance of a string for the given push down automata (acceptance by final states)

In [ ]:
# 10b

## Author : Arjun ##

class PDA:
    def __init__(self, states, input_alphabet, stack_alphabet, transition_function, start_state, start_stack_symbol, accept_states):
        self.states = states
        self.input_alphabet = input_alphabet
        self.stack_alphabet = stack_alphabet
        self.transition_function = transition_function  # Dictionary of (state, input_symbol, stack_top) -> [(state, new_stack_top)]
        self.start_state = start_state
        self.start_stack_symbol = start_stack_symbol
        self.accept_states = accept_states

    def accepts(self, input_string):
        # Initialize the stack and start configuration
        initial_config = (self.start_state, [self.start_stack_symbol])
        configurations = [initial_config]

        # Process each symbol in the input string
        for symbol in input_string:
            new_configurations = []
            for (state, stack) in configurations:
                # Check if there's a transition available for the current input and stack top
                stack_top = stack[-1] if stack else None
                if (state, symbol, stack_top) in self.transition_function:
                    transitions = self.transition_function[(state, symbol, stack_top)]
                    for (next_state, stack_change) in transitions:
                        new_stack = stack[:-1]  # Pop the stack top
                        if stack_change != "":
                            new_stack += stack_change[::-1]  # Push new symbols to the stack
                        new_configurations.append((next_state, new_stack))
            configurations = new_configurations

        # Check if any configuration is in an accept state
        return any(state in self.accept_states and len(input_string) == 0 for (state, stack) in configurations)

# Example Usage
pda = PDA(
    states={'q0', 'q1', 'q2'},
    input_alphabet={'0', '1'},
    stack_alphabet={'Z', '0'},
    transition_function={
        ('q0', '0', 'Z'): [('q1', '0Z')],
        ('q1', '0', '0'): [('q1', '00')],
        ('q1', '1', '0'): [('q2', '')],
        ('q2', '1', '0'): [('q2', '')],
    },
    start_state='q0',
    start_stack_symbol='Z',
    accept_states={'q2'}
)

input_string = "0011"
result = pda.accepts(input_string)
print(f"The input string '{input_string}' is " + ("accepted" if result else "not accepted") + " by the PDA.")